In [103]:
import numpy as np 
import pims
import skimage as ski 
import matplotlib as mpl
import matplotlib.pyplot as plt 
%matplotlib inline
import pandas as pd
import trackpy as tp
from trackpy import predict
import pickle
import linking_2d as l2d
import time 
import visualization as vs
#tweak plot styles
mpl.rc('figure',  figsize=(15, 5))
mpl.rc('image', cmap='gray')
tp.ignore_logging() # important to not spam too much print output 

In [104]:
#run these if you want to reload imported modules every time the code is run 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [135]:
# load in features
tL = pd.read_hdf('/media/kpierce/Data/steadyspt4.1-rectified/analysis/trajectories/leftfeatures.h5')
tR = pd.read_hdf('/media/kpierce/Data/steadyspt4.1-rectified/analysis/trajectories/rightfeatures.h5')

# increase y coordinate of every feature by 245 to undo the cropping effect
tL['y']+=245
tR['y']+=245

In [136]:
# define the predictor 
@tp.predict.predictor
def predict(t1, particle):
    velocity = np.array((5, 0)) 
    return particle.pos + velocity * (t1 - particle.t)

In [6]:
# link features into 2d trajectories 
tL1 = tp.link_df(tL, search_range=50,memory=15, predictor=predict)
tL1.to_hdf('/media/kpierce/Data/steadyspt4.1-rectified/analysis/trajectories/leftlinkedfeatures.h5',key='df',mode='w')

In [7]:
# link right features in 2d trajectories 
tR1 = tp.link_df(tR, search_range=50,memory=15, predictor=predict)
tR1.to_hdf('/media/kpierce/Data/steadyspt4.1-rectified/analysis/trajectories/rightlinkedfeatures.h5',key='df',mode='w')

In [137]:
# or skip these steps and load the linked features ... 
#tL1 = pd.read_hdf('/media/kpierce/Data/steadyspt4.1-rectified/analysis/trajectories/leftlinkedfeatures.h5')
#tR1 = pd.read_hdf('/media/kpierce/Data/steadyspt4.1-rectified/analysis/trajectories/rightlinkedfeatures.h5')

In [138]:
# create trajectory objects from the raw data 
tL2 = l2d.generate_trajectories(tL1,'L',minlength=20) #minlength means number of time points involved in the trajectory
tR2 = l2d.generate_trajectories(tR1,'R',minlength=20) 

In [139]:
# filter trajectories by their travel distance .. 30 pixels is about 1 particle diameter 
tL3 = l2d.distance_filter(tL2)
tR3 = l2d.distance_filter(tR2)

In [140]:
# save trajectories
f = open('../trajectories/tL-pickle', 'wb')
pickle.dump(tL3, f)
f.close()
f = open('../trajectories/tR-pickle', 'wb')
pickle.dump(tR3, f)
f.close()

In [141]:
# now interpolate trajectories
# get the left one 
tL4 = l2d.interp_trajectories(tL3,'L')
# save interpolated trajectories 
f = open('../trajectories/tLi-pickle', 'wb')
pickle.dump(tL4, f)
f.close()

In [142]:
# and the right one 
tR4 = l2d.interp_trajectories(tR3,'R')
# save interpolated trajectories 
f = open('../trajectories/tRi-pickle', 'wb')
pickle.dump(tR4, f)
f.close()